In [1]:
import pandas as pd,\
  seaborn as sns,\
  matplotlib.pyplot as pypl,\
  numpy as np

#  plotly.express as px,\

In [2]:
pypl.style.use('bmh')

In [3]:
df_historical_pulls = pd.read_csv(
  '../dba5101_gp3/data/pulls_output.csv',
  header='infer',
  index_col=False
)

In [4]:
# Add in historical pulls over all time

df_total_pulls = df_historical_pulls.groupby(['arm'])['global_pull'].count().reset_index()

df_total_pulls.rename(columns={"global_pull": "total_pulls"},inplace=True)

df_historical_pulls =  df_historical_pulls.merge(right =df_total_pulls , how ='left', on ='arm' )

In [5]:
df_arms = pd.DataFrame( {'arm': df_historical_pulls['arm'].unique().tolist()  } )

df_pulls = pd.DataFrame( {'global_pull': df_historical_pulls['global_pull'].unique().tolist()  } )

df_cartesian = df_arms.merge(right=df_pulls, how='cross')

df_cartesian= df_cartesian.merge( right=df_historical_pulls, on=['global_pull','arm'] ,how='left' )

df_cartesian['arm_reward'].fillna(value=0,inplace=True)

df_cartesian['cumulative_reward'] = df_cartesian.groupby(['arm'])['arm_reward'].cumsum(skipna=True)

df_cartesian['arm_pull'].ffill(inplace=True)

In [6]:


df_cartesian = df_cartesian[df_cartesian['global_pull'] >= 24]

In [7]:
#df_cartesian['ucb_index'] = 
df_cartesian['contemporary_linucb'] = (df_cartesian['cumulative_reward'] / (df_cartesian['arm_pull'] + 1) )  + np.sqrt(
  (2 * np.log( df_cartesian['global_pull']  ) ) / (df_cartesian['arm_pull'] + 1)
)

df_cartesian['ucb_mean'] = (df_cartesian['cumulative_reward'] / (df_cartesian['arm_pull'] + 1) )

df_cartesian['penalty'] = np.sqrt(
  (2 * np.log( df_cartesian['global_pull']  ) ) / (df_cartesian['arm_pull'] + 1)
)

In [8]:
sns.lineplot(
  x='global_pull',
  y='contemporary_linucb',
  hue='arm',
  data=df_cartesian[
    (df_cartesian['global_pull'] <= 220) 
    & (df_cartesian['global_pull'] >= 24) 
    & (df_cartesian['arm'].isin([0,3,8,23]) ) 
  ]
)

pypl.title("UCB Over Time")

pypl.gcf().set_size_inches(17, 10)

pypl.ylabel('UCB Index')
pypl.xlabel('Pull')

#pypl.show()

pypl.savefig(fname='./ucb_exploitation_line.png')

pypl.close('all')

In [9]:
df_cartesian[
  df_cartesian['global_pull'] == 2020
].sort_values(
  by=['contemporary_linucb'],
  ascending=False
)[['arm','arm_pull','cumulative_reward','contemporary_linucb','ucb_mean','penalty']].head(n=5)

,arm,arm_pull,cumulative_reward,contemporary_linucb,ucb_mean,penalty
2020,0.0,1822.0,44651.0,24.584520,24.493143,0.091377
16188,7.0,0.0,19.0,22.901500,19.000000,3.901500
42500,20.0,0.0,19.0,22.901500,19.000000,3.901500
20236,9.0,0.0,19.0,22.901500,19.000000,3.901500
36428,17.0,1.0,40.0,22.758777,20.000000,2.758777


In [10]:
sns.lineplot(
  x='global_pull',
  y='contemporary_linucb',
  hue='arm',
#  color='b',
  data=df_cartesian[
    (df_cartesian['global_pull'] >= 24) 
    & (df_cartesian['arm'].isin([0,7,17]) ) 
  ]
)

pypl.title("UCB Over Time - Candidates For Next Arm, Due To Uncertainity")

pypl.gcf().set_size_inches(17, 10)

pypl.ylabel('UCB Index')
pypl.xlabel('Pull')

#pypl.show()
pypl.savefig(fname='./ucb_future_line.png')

pypl.close('all')

In [11]:
f, ax = pypl.subplots(figsize=(17, 10))

sns.lineplot(
  x='global_pull',
  y='ucb_mean',
  hue='arm',
#  color='b',
  data=df_cartesian[
    (df_cartesian['global_pull'] >= 24) 
    & (df_cartesian['arm'].isin([0,7,17]) ) 
  ]
)

sns.lineplot(
  x='global_pull',
  y='penalty',
  hue='arm',
  data=df_cartesian[
    (df_cartesian['global_pull'] >= 24) 
    & (df_cartesian['arm'].isin([0,7,17]) ) 
  ]
)


pypl.title("Arm Mean & Penalty Term Over Time")

pypl.ylabel('UCB Index Component Quantity')
pypl.xlabel('Pull')


#pypl.show()
pypl.savefig(fname='./ucb_penalty_line.png')

pypl.close('all')

In [12]:
#Create combo chart
fig, ax1 = pypl.subplots(figsize=(17,10))


#bar plot creation
ax1.set_title('UCB Index & Reward - Exploitation Phase', fontsize=16)
ax1.set_xlabel('Pull')
ax1.set_ylabel('Reward')
ax1 = sns.scatterplot(x='global_pull', y='arm_reward', hue='arm', data = df_historical_pulls[
(df_historical_pulls['global_pull'] <= 250) 
& (df_historical_pulls['global_pull'] >= 24) 
#& (df_historical_pulls['arm'].isin([0,3,8,23]) ) 
])
ax1.tick_params(axis='y')
#specify we want to share the same x-axis
ax2 = ax1.twinx()

#line plot creation
ax2.set_ylabel('UCB Index')
ax2 = sns.lineplot(
  x='global_pull',
  y='contemporary_linucb',
  hue='arm',
  data=df_cartesian[
    (df_cartesian['global_pull'] <= 250) 
    & (df_cartesian['global_pull'] >= 24) 
#    & (df_cartesian['arm'].isin([0,3,8,23]) ) 
  ]
)


#show plot
#pypl.show()
pypl.savefig(fname='./ucb_index_reward_line.png')

pypl.close('all')

In [48]:
#sns.scatterplot( x='global_pull', y='arm_reward',   hue='arm', 
#  data = df_historical_pulls[   (df_historical_pulls['global_pull'] <= 250) 
#    & (df_historical_pulls['global_pull'] >= 24) ])

#pypl.gcf().set_size_inches(17, 10)

#pypl.show()

#pypl.close('all')

In [47]:
#sns.displot(  data=df_historical_pulls[ df_historical_pulls['global_pull'] >= 24] , 
#  x="arm_reward", hue="arm", kind="kde")

#pypl.gcf().set_size_inches(17, 10)

#pypl.show()

#pypl.close('all')

In [13]:
sns.violinplot(
  cut=0,
  inner='quartiles',
  data=df_historical_pulls[ (df_historical_pulls['global_pull'] >= 24) & (df_historical_pulls[ 'total_pulls'] > 2 ) ] , 
  x='arm',
  y='arm_reward',
)

pypl.gcf().set_size_inches(17, 10)
pypl.ylim([0,45])

pypl.title("Arm Reward Violinplot, Exploitation Phase - With Quartiles")
pypl.ylabel('Reward')
pypl.xlabel('Arm')

#pypl.show()
pypl.savefig(fname='./reward_violin.png')

pypl.close('all')